# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as convolutional neural networks (CNNs).

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tf_ds

## Data

That's where we load and preprocess our data.

In [9]:
mnist_dataset, mnist_info = tf_ds.load(name='mnist', data_dir='D:\\Udemy\\The Data Science Course 2021\\PythonCodeML\\dataset', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [13]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose the optimizer and the loss function

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [15]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3349 - accuracy: 0.9058 - val_loss: 0.1744 - val_accuracy: 0.9485 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1406 - accuracy: 0.9577 - val_loss: 0.1203 - val_accuracy: 0.9647 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1005 - accuracy: 0.9693 - val_loss: 0.0939 - val_accuracy: 0.9715 - 2s/epoch - 4ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0797 - accuracy: 0.9752 - val_loss: 0.0895 - val_accuracy: 0.9745 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0643 - accuracy: 0.9801 - val_loss: 0.0651 - val_accuracy: 0.9792 - 2s/epoch - 4ms/step


## Test the model

In [16]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 626ms/step - loss: 0.0857 - accuracy: 0.9726


In [18]:
print('Test loss {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss 0.09. Test accuracy: 97.26%
